In [37]:
import time
import wings
from wings.planner import Planner
from wings.component import Component
from wings.execution import Execution

In [38]:
SERVER = "http://localhost:9090/wings-portal"
INTERNAL_SERVER = SERVER

In [39]:
USERID = "admin"
PASSWORD = "admin123"
DOMAIN = "CaesarCypherParallel"

In [40]:
TESTS_CAESAR = [
    {
        "template": "CaesarCypherMap",
        "inputs": {
            "Document": "file:TwelveDaysOfChristmas.txt",
            "ShiftKey": 3,
            "numberOfChunks": 4
        },
        "outputs": {
            "planner": {
                "expansions": [
                    {
                        "components": {
                            "breakDoc": 1,
                            "Caesar": 4
                        },
                        "execute": True
                    }

                ]
            }
        }
    }
]

In [41]:
TESTS_PALEOTS = [
    {
        "template": "Test_Spectral",
        "inputs": {
            "InputTS": "file:test_perfect_signal.json",
            "freq_method": "log",
            "qs": 0.95
        },
        "outputs": {
            "planner": {
                "expansions": [
                    {
                        "components": {
                            "cwt-psd": 1,
                            "AR1sim": 1
                        },
                        "execute": True
                    },
                    {
                        "components": {
                            "MTM": 1,
                            "AR1sim": 1
                        },
                        "execute": True
                    },
                    {
                        "components": {
                            "Welch": 1,
                            "AR1sim": 1
                        },
                        "execute": True
                    },
                    {
                        "components": {
                            "Periodogram": 1,
                            "AR1sim": 1
                        },
                        "execute": True
                    }
                ]
            }
        }
    },
    {
        "template": "Test_Spectral",
        "inputs": {
            "InputTS": "file:test_noisy_signal.json",
            "freq_method": "log",
            "qs": 0.95
        },
        "outputs": {
            "planner": {
                "expansions": [
                    {
                        "components": {
                            "wwz-psd": 1,
                            "AR1sim": 1
                        },
                        "execute": True
                    },
                    {
                        "components": {
                            "LombScargle": 1,
                            "AR1sim": 1
                        },
                        "execute": True
                    }
                ]
            }
        }
    },
    {
        "template": "Test_Wavelet",
        "inputs": {
            "InputTS": "file:test_perfect_signal.json",
            "freq_method": "log",
            "qs": 0.95
        },
        "outputs": {
            "planner": {
                "expansions": [
                    {
                        "components": {
                            "cwt": 1,
                            "AR1sim-wave": 1
                        },
                        "execute": True
                    }
                ]
            }
        }
    },
    {
        "template": "Test_Wavelet",
        "inputs": {
            "InputTS": "file:test_noisy_signal.json",
            "freq_method": "log",
            "qs": 0.95
        },
        "outputs": {
            "planner": {
                "expansions": [
                    {
                        "components": {
                            "cwt": 1,
                            "AR1sim-wave": 1
                        },
                        "execute": True
                    }
                ]
            }
        }
    }
]

In [42]:
def getTemplateComponents(expansion):
    template = expansion["template"]
    components = {}
    for nodeid in template["Nodes"]:
        cvarId = template["Nodes"][nodeid]["componentVariable"]["binding"]["id"]
        cvar = cvarId.split("#")[-1]
        cvarnum = components.get(cvar, 0)
        cvarnum += 1
        components[cvar] = cvarnum
    return components

In [43]:
def matches_test_expansion(expansion, test_expansions):
    components = getTemplateComponents(expansion)
    print("\t " + str(components))
    for test_expansion in test_expansions:
        if components == test_expansion["components"]:
            return test_expansion
    return None

In [44]:
api = wings.ApiClient(server=SERVER, username=USERID, password=PASSWORD)
api.set_domain(DOMAIN)

In [45]:
# Clean the Component Cache
# FIXME: Not cleaning right now to avoid data pollution while testing
# - Need a way to clean the test data after tests are done

#components = Component(api)
#components.increment_versions_clear_cache()

In [46]:
# Plan and check
planner = Planner(api)
runids = []

for test in TESTS_PALEOTS:
    print("Testing template: " + test['template'])
    test_expansions = test["outputs"]["planner"]["expansions"]

    print("\t Planning workflow expansions")
    planner.set_template(test["template"])
    plan = planner.get_expansions(test["inputs"])
    if plan["success"]:
        expansions = plan["data"]["templates"]
        seed = plan["data"]["seed"]
        if len(test_expansions) != len(expansions):
            print("\t\t FAILED: Expected %d expanded workflows, got %d" % (len(test_expansions), len(expansions)))
            continue

        for expansion in expansions:
            print("\t Testing expansion")
            test_expansion = matches_test_expansion(expansion, test_expansions)
            if test_expansion is not None:
                print("\t\t Expanded workflow components match expected")                
                if "execute" in test_expansion and test_expansion["execute"]:
                    print("\t\t Submitting for execution")
                    runid = planner.run_workflow(expansion, seed)
                    runids.append(runid)
            else:
                print("\t\t FAILED: Expanded workflow components do not match expected")
    else:
        print("\t\t FAILED: Planning failed")
        print(plan)


Testing template: Test_Spectral
	 Planning workflow expansions
	 Testing expansion
	 {'AR1sim': 1, 'cwt-psd': 1}
		 Expanded workflow components match expected
		 Submitting for execution
	 Testing expansion
	 {'AR1sim': 1, 'Welch': 1}
		 Expanded workflow components match expected
		 Submitting for execution
	 Testing expansion
	 {'MTM': 1, 'AR1sim': 1}
		 Expanded workflow components match expected
		 Submitting for execution
	 Testing expansion
	 {'AR1sim': 1, 'Periodogram': 1}
		 Expanded workflow components match expected
		 Submitting for execution
Testing template: Test_Spectral
	 Planning workflow expansions
	 Testing expansion
	 {'AR1sim': 1, 'wwz-psd': 1}
		 Expanded workflow components match expected
		 Submitting for execution
	 Testing expansion
	 {'LombScargle': 1, 'AR1sim': 1}
		 Expanded workflow components match expected
		 Submitting for execution
Testing template: Test_Wavelet
	 Planning workflow expansions
	 Testing expansion
	 {'AR1sim-wave': 1, 'cwt': 1}
		 Expand

In [47]:
execution_api = Execution(api)
runs_finished = 0
runs_successful = 0

print("Monitoring runs...")
while runs_finished < len(runids):
    time.sleep(10)

    for runid in runids:
        details = execution_api.get_run_details(runid)
        run_status = details["execution"]["runtimeInfo"]["status"]
        if run_status == "SUCCESS":
            runs_finished += 1
            runs_successful += 1
            print(f"\t\t Run {runid} finished successfully")
        elif run_status == "FAILURE":
            runs_finished += 1
            print(f"\t\t FAILED: Run {runid} failed")

print("Runs successful: %d out of %d" % (runs_successful, len(runids)))

Monitoring runs...
		 Run Test_Spectral-ad-478289d5-6bae-4ad5-8be7-d32b0926f78b finished successfully
		 Run Test_Spectral-ad-dec24b38-fb11-43fa-bf24-b2f2af49c3df finished successfully
		 Run Test_Spectral-ad-55cdbd6d-f583-4b6c-990a-585e926d9e44 finished successfully
		 Run Test_Spectral-ad-3db8bf34-f1db-40aa-b0bc-42bbcc2a2c82 finished successfully
		 FAILED: Run Test_Spectral-72-fe1a32a5-bb23-4de7-8641-89ff1af230c5 failed
		 Run Test_Spectral-72-c68f91e8-65c2-4254-8060-14448614504d finished successfully
		 Run Test_Wavelet-b9d-9b34156a-1fd5-4d95-960f-c382e95dd99c finished successfully
Runs successful: 6 out of 7


In [48]:
api.logout()